<a href="https://colab.research.google.com/github/letruong711/Neural-Networks-and-Pneumonia/blob/main/Pneumonia_and_Talos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Libraries.**

In [ ]:
#get more RAM in this notebook
[1]*9**10

In [ ]:
!pip install talos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import os
import glob
import random

import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist
from keras.models import load_model
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers

import datetime

import talos

# **Importing Data.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#setting up bridge between collab and drive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
fid = drive.ListFile({'q': "title='archive.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id':fid})
f.GetContentFile('archive.zip')

In [ ]:
#unzip massive file.
!unzip archive.zip

In [ ]:
PATH = '/content/chest_xray'

In [ ]:
train_dir = os.path.join(PATH, 'train')
test_dir = os.path.join(PATH, 'test')
val_dir = os.path.join(PATH, 'val')

# **Preprocessing.**

In [ ]:
#create function that helps us visualize our model
def visualize_training_results(results):
    history = results.history
    plt.figure()
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    plt.figure()
    plt.plot(history['val_accuracy'])
    plt.plot(history['accuracy'])
    plt.legend(['val_acc_accuracy', 'accuracy'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()

In [ ]:
#set image dimensions
IMG_HEIGHT = 224
IMG_WIDTH = 224
batch_size = 624

In [ ]:
image_gen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_data_gen = image_gen.flow_from_directory(
    directory=train_dir,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size= 5216,
    class_mode='binary')

Found 5216 images belonging to 2 classes.


In [ ]:
test_data_gen = image_gen.flow_from_directory(
    directory = test_dir,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = batch_size,
    class_mode='binary')

Found 624 images belonging to 2 classes.


In [ ]:
#creating X and y test, train datasets
X_train, y_train = next(train_data_gen)
X_test, y_test = next(test_data_gen)

In [ ]:
# Confirming the shapes and types of the datasets

display(np.shape(X_train))
display(np.shape(y_train))

display(type(X_train))
display(type(y_train))

print('-------------')

display(np.shape(X_test))
display(np.shape(y_test))

display(type(X_test))
display(type(y_test))

(5216, 224, 224, 3)

(5216,)

numpy.ndarray

numpy.ndarray

-------------


(624, 224, 224, 3)

(624,)

numpy.ndarray

numpy.ndarray

# **Setting Up Talos.**

In [ ]:
#in order to tune our CNN, we need to figure out what parameters work the best 
#Below are the parameters I've chosen:
params = {'dropout': [0.1, 0.3, 0.5],
          'optimizer': ['adam', 'sgd'],
          'activation1': ['relu', 'tanh']}

In [ ]:
#use generic CNN model with parameters options.
def tuned_model(X_train, y_train, X_test, y_test, params):
    model = Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation=params['activation1'], input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), padding='same'))
    model.add(layers.MaxPool2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation=params['activation1'], padding='same'))
    model.add(layers.MaxPool2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation=params['activation1'], padding='same'))
    model.add(layers.MaxPool2D(pool_size=(3, 3)))
    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation=params['activation1']))
    model.add(layers.Dense(60, activation=params['activation1']))
    model.add(layers.Dropout(params['dropout']))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=params['optimizer'], metrics='accuracy')
    es = EarlyStopping(patience=20, monitor= 'val_accuracy', restore_best_weights=True)

    out = model.fit(X_train, 
                y_train,
                batch_size = 50,
                epochs = 200,
                validation_data=(X_test, y_test),
                callbacks = es)

    return out, model

In [ ]:
results = talos.Scan(X_train, y_train, params=params, model=tuned_model, experiment_name='grid')



  0%|          | 0/12 [00:00<?, ?it/s]

Epoch 1/200
74/74 [==============================] - 113s 2s/step - loss: 0.2806 - accuracy: 0.8847 - val_loss: 0.1925 - val_accuracy: 0.9335
Epoch 2/200
74/74 [==============================] - 112s 2s/step - loss: 0.1147 - accuracy: 0.9556 - val_loss: 0.0934 - val_accuracy: 0.9693
Epoch 3/200
74/74 [==============================] - 112s 2s/step - loss: 0.0903 - accuracy: 0.9669 - val_loss: 0.1403 - val_accuracy: 0.9450
Epoch 4/200
74/74 [==============================] - 112s 2s/step - loss: 0.0661 - accuracy: 0.9762 - val_loss: 0.1201 - val_accuracy: 0.9629
Epoch 5/200
74/74 [==============================] - 111s 2s/step - loss: 0.0539 - accuracy: 0.9792 - val_loss: 0.0999 - val_accuracy: 0.9719
Epoch 6/200
74/74 [==============================] - 112s 2s/step - loss: 0.0359 - accuracy: 0.9877 - val_loss: 0.1310 - val_accuracy: 0.9610
Epoch 7/200
74/74 [==============================] - 111s 1s/step - loss: 0.0460 - accuracy: 0.9830 - val_loss: 0.1459 - val_accuracy: 0.9674
Epoch 

  0%|          | 0/12 [1:58:37<?, ?it/s]


  8%|▊         | 1/12 [1:56:54<21:25:56, 7014.23s/it]

Epoch 1/200
74/74 [==============================] - 118s 2s/step - loss: 0.5701 - accuracy: 0.7373 - val_loss: 0.7958 - val_accuracy: 0.2562
Epoch 2/200
74/74 [==============================] - 113s 2s/step - loss: 0.5339 - accuracy: 0.7450 - val_loss: 1.0492 - val_accuracy: 0.2562
Epoch 3/200
74/74 [==============================] - 114s 2s/step - loss: 0.4668 - accuracy: 0.7798 - val_loss: 0.8448 - val_accuracy: 0.2562
Epoch 4/200
74/74 [==============================] - 114s 2s/step - loss: 0.4501 - accuracy: 0.8061 - val_loss: 0.5057 - val_accuracy: 0.7527
Epoch 5/200
74/74 [==============================] - 113s 2s/step - loss: 0.3978 - accuracy: 0.8425 - val_loss: 0.6904 - val_accuracy: 0.7463
Epoch 6/200
74/74 [==============================] - 113s 2s/step - loss: 0.3125 - accuracy: 0.8746 - val_loss: 0.2468 - val_accuracy: 0.8831
Epoch 7/200
74/74 [==============================] - 113s 2s/step - loss: 0.2668 - accuracy: 0.8995 - val_loss: 0.1710 - val_accuracy: 0.9374
Epoch 



 17%|█▋        | 2/12 [3:01:04<14:18:52, 5153.21s/it]

...



 83%|████████▎ | 10/12 [16:50:41<3:26:23, 6191.63s/it]

Epoch 1/200
74/74 [==============================] - 119s 2s/step - loss: 0.7276 - accuracy: 0.6743 - val_loss: 0.5718 - val_accuracy: 0.7438
Epoch 2/200
74/74 [==============================] - 117s 2s/step - loss: 0.6392 - accuracy: 0.7009 - val_loss: 0.5741 - val_accuracy: 0.7438
Epoch 3/200
74/74 [==============================] - 117s 2s/step - loss: 0.6316 - accuracy: 0.7127 - val_loss: 0.5799 - val_accuracy: 0.7438
Epoch 4/200
74/74 [==============================] - 117s 2s/step - loss: 0.6261 - accuracy: 0.7050 - val_loss: 0.5805 - val_accuracy: 0.7438
Epoch 5/200
74/74 [==============================] - 118s 2s/step - loss: 0.5999 - accuracy: 0.7321 - val_loss: 0.5784 - val_accuracy: 0.7438
Epoch 6/200
74/74 [==============================] - 117s 2s/step - loss: 0.5984 - accuracy: 0.7390 - val_loss: 0.5691 - val_accuracy: 0.7438
Epoch 7/200
74/74 [==============================] - 117s 2s/step - loss: 0.6007 - accuracy: 0.7250 - val_loss: 0.5727 - val_accuracy: 0.7438
Epoch 



 92%|█████████▏| 11/12 [17:31:54<1:24:13, 5053.52s/it]

Epoch 1/200
74/74 [==============================] - 119s 2s/step - loss: 0.4848 - accuracy: 0.7757 - val_loss: 1.7059 - val_accuracy: 0.2562
Epoch 2/200
74/74 [==============================] - 117s 2s/step - loss: 0.3932 - accuracy: 0.8442 - val_loss: 0.2384 - val_accuracy: 0.9182
Epoch 3/200
74/74 [==============================] - 117s 2s/step - loss: 0.2416 - accuracy: 0.9088 - val_loss: 1.4253 - val_accuracy: 0.3304
Epoch 4/200
74/74 [==============================] - 117s 2s/step - loss: 0.2243 - accuracy: 0.9189 - val_loss: 0.1421 - val_accuracy: 0.9508
Epoch 5/200
74/74 [==============================] - 119s 2s/step - loss: 0.1504 - accuracy: 0.9444 - val_loss: 0.1244 - val_accuracy: 0.9597
Epoch 6/200
74/74 [==============================] - 117s 2s/step - loss: 0.1334 - accuracy: 0.9469 - val_loss: 0.1104 - val_accuracy: 0.9674
Epoch 7/200
74/74 [==============================] - 117s 2s/step - loss: 0.1151 - accuracy: 0.9570 - val_loss: 0.1494 - val_accuracy: 0.9374
Epoch 



100%|██████████| 12/12 [18:33:16<00:00, 5566.41s/it]


In [ ]:
results.best_model(metric='val_accuracy')

In [ ]:
pd.read_csv('/content/grid/112421165950.csv').sort_values('val_accuracy', ascending=False)

,round_epochs,loss,accuracy,val_loss,val_accuracy,activation1,dropout,optimizer
0,63,0.000002,1.000000,0.282803,0.978275,relu,0.1,adam
4,54,0.000121,1.000000,0.291317,0.975719,relu,0.5,adam
5,98,0.006258,0.998904,0.110883,0.975719,relu,0.5,sgd
3,81,0.070128,0.978636,0.068410,0.975080,relu,0.3,sgd
9,76,0.009177,0.999178,0.084064,0.975080,tanh,0.3,sgd
2,30,0.000031,1.000000,0.237178,0.973163,relu,0.3,adam
11,31,0.052863,0.981101,0.080413,0.971885,tanh,0.5,sgd
1,34,0.100795,0.961107,0.116134,0.956550,relu,0.1,sgd
7,51,0.023927,0.991235,0.257820,0.916933,tanh,0.1,sgd
6,21,0.575373,0.742536,0.569230,0.743770,tanh,0.1,adam
